## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-25-02-17-45 +0000,startribune,Napheesa Collier shines in return from injury ...,https://www.startribune.com/napheesa-collier-s...
1,2025-08-25-02-11-44 +0000,nyt,National Guard Patrols Begin to Carry Weapons ...,https://www.nytimes.com/2025/08/24/us/politics...
2,2025-08-25-02-00-00 +0000,wsj,Demand for workers has cooled yet the unemploy...,https://www.wsj.com/economy/immigration-worker...
3,2025-08-25-01-56-00 +0000,wsj,Here’s what you need to know about the unusual...,https://www.wsj.com/politics/policy/texas-cali...
4,2025-08-25-01-53-02 +0000,startribune,"Vikings sign QB Carson Wentz, trade Sam Howell...",https://www.startribune.com/minnesota-vikings-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2329/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
122,trump,23
507,ukraine,12
63,new,12
266,russia,9
366,war,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
90,2025-08-24-18-08-34 +0000,nypost,Russia’s top diplomat says Zelensky can’t sign...,https://nypost.com/2025/08/24/world-news/russi...,60
110,2025-08-24-14-39-41 +0000,nypost,Trump announces he’ll attend Ryder Cup in New ...,https://nypost.com/2025/08/24/us-news/trump-an...,59
89,2025-08-24-18-08-34 +0000,nypost,Vance reveals what Putin has conceded in Ukrai...,https://nypost.com/2025/08/24/us-news/vance-re...,54
121,2025-08-24-14-05-19 +0000,nypost,Russia’s foreign minister throws up a new cond...,https://nypost.com/2025/08/24/us-news/russias-...,53
76,2025-08-24-19-42-25 +0000,nyt,Trump Says Chicago and New York Are Next for F...,https://www.nytimes.com/2025/08/22/us/politics...,52


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
90,60,2025-08-24-18-08-34 +0000,nypost,Russia’s top diplomat says Zelensky can’t sign...,https://nypost.com/2025/08/24/world-news/russi...
76,52,2025-08-24-19-42-25 +0000,nyt,Trump Says Chicago and New York Are Next for F...,https://www.nytimes.com/2025/08/22/us/politics...
86,37,2025-08-24-18-30-00 +0000,nypost,Australian mom accused of killing American ste...,https://nypost.com/2025/08/24/world-news/lisa-...
3,29,2025-08-25-01-56-00 +0000,wsj,Here’s what you need to know about the unusual...,https://www.wsj.com/politics/policy/texas-cali...
63,29,2025-08-24-20-46-12 +0000,nypost,‘Sex and the City’ fans honor ‘iconic’ star Ki...,https://nypost.com/2025/08/24/us-news/sex-and-...
121,27,2025-08-24-14-05-19 +0000,nypost,Russia’s foreign minister throws up a new cond...,https://nypost.com/2025/08/24/us-news/russias-...
72,26,2025-08-24-20-14-54 +0000,nypost,Florida sheriff shows off $50K chain snatched ...,https://nypost.com/2025/08/24/us-news/florida-...
140,25,2025-08-24-10-24-22 +0000,nypost,Russia accuses Kyiv of striking nuclear power ...,https://nypost.com/2025/08/24/world-news/russi...
6,22,2025-08-25-01-35-57 +0000,nypost,Freak hot tub incident almost kills 2 elderly ...,https://nypost.com/2025/08/24/us-news/freak-ho...
101,22,2025-08-24-16-41-00 +0000,wsj,President Trump’s attempts to control monetary...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
